In [11]:
import arcpy
import logging
from datetime import date

# Set the path to your geodatabase
geodatabase_path = "D:\\GDB_Connections\\cs_gdb1_sde.sde"
logFileLocation = r'D:\temp\GDBMaintenanceLog'
reconcilelogFileLocation = r'D:\temp\reconcilelog_{}.txt'.format(date.today().strftime("_%d_%m_%Y"))
targetVersion = 'sde.DEFAULT'

# Setup logging
logging.basicConfig(filename=logFileLocation + date.today().strftime("_%d_%m_%Y") + '.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(levelname)s %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        level=logging.INFO)

logging.info("Reconcile script starting")

try:
    # Check if the geodatabase exists
    if arcpy.Exists(geodatabase_path):
        #Sometimes a stale connection can cause issues with versioning.
        arcpy.ClearWorkspaceCache_management()

        # Block new connections to the database.
        logging.info("The database is no longer accepting connections")
        arcpy.AcceptConnections(geodatabase_path, False)

        # Disconnect all users from the database.
        logging.info("Disconnecting all users")
        arcpy.DisconnectUser(geodatabase_path, "ALL")
        
        # Get a list of versions to pass into the ReconcileVersions tool.
        # Only reconcile versions that are children of Default
        logging.info("Compiling a list of versions to reconcile")
        verList = arcpy.da.ListVersions(geodatabase_path)
        versionList = [ver.name for ver in verList if ver.parentVersionName == targetVersion]
        print (versionList)
        
        # Execute the ReconcileVersions tool.
        # We are not posting any edits to default or deleting any versions here
        logging.info("Reconciling all versions")
        arcpy.ReconcileVersions_management(geodatabase_path, "ALL_VERSIONS", targetVersion, versionList, "LOCK_ACQUIRED",
                                           "NO_ABORT", "BY_OBJECT", "FAVOR_EDIT_VERSION", "NO_POST", "KEEP_VERSION",
                                           reconcilelogFileLocation, 'PROCEED', 'RECONCILE')
        
         # Allow the database to begin accepting connections again
        logging.info("Allow users to connect to the database again")
        arcpy.AcceptConnections(geodatabase_path, True)
        
except Exception as e:
    print(str(e))
    print(arcpy.GetMessages(2))
    logging.error(arcpy.GetMessages(2))
    logging.error(str(e))

finally:
    logging.info("Script completed")
    print("Done.")

['SDE.test', 'SDE.New Version']
Done.
